In [ ]:
# Cell 1 - Imports and Data Loading
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Load Data
data = pd.read_csv('data.csv')


In [ ]:
# Cell 2 - Correlation Matrix Heatmap
corr = data.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Cell 3 - Lag Correlation Analysis
for lag in range(1, 11):
    lag_correlation = data['variable'].autocorr(lag=lag)
    print(f'Lag {lag}: {lag_correlation}')


In [ ]:
# Cell 4 - Cross-Correlation Plots
sm.tsa.graphics.tsa_plot(data['variable1'], data['variable2'], lags=40)
plt.show()


In [ ]:
# Cell 5 - Event Detection Algorithm with Threshold
def detect_events(data, threshold):
    return data[data['value'] > threshold]

detected_events = detect_events(data, threshold=10)
detected_events


In [ ]:
# Cell 6 - Storm Catalog Generation
storm_catalog = detected_events.groupby('event_id').agg({'timestamp': 'min', 'value': 'max'})
storm_catalog


In [ ]:
# Cell 7 - Statistical Significance Testing with P-Values
from scipy import stats
p_values = stats.norm.sf(abs(storm_catalog['value'])) * 2
p_values


In [ ]:
# Cell 8 - Visualization of Detected Events
plt.figure(figsize=(12, 6))
plt.plot(data['timestamp'], data['value'], label='Value')
plt.scatter(detected_events['timestamp'], detected_events['value'], color='red', label='Detected Events')
plt.title('Detected Events')
plt.legend()
plt.show()


In [ ]:
# Cell 9 - Summary Statistics
summary_statistics = data.describe()
summary_statistics
